In this final phase of my project, I constructed a fully functional wage forecasting system, encapsulated in a user-friendly graphical user interface (GUI) using Tkinter. This system integrates the predictive modeling capabilities developed in earlier stages with an interface that allows users to input their details and receive personalized wage predictions. Here’s a detailed breakdown of the system components and functionalities:

1. **Data Preparation and Model Initialization**:
   - I load and preprocess the wage data, filling missing values and splitting into training and test datasets.
   - The data is scaled using a MinMaxScaler to normalize the feature values, crucial for the performance of the XGBoost model.
   - I initialize the XGBoost model with the optimized parameters identified from previous steps and train it on the scaled training data.

2. **GUI Setup**:
   - The main window (`root`) of the application is created using Tkinter.
   - Various input fields are set up for users to enter their details, including dropdown menus for categorical choices like employment category, gender, and marital status.
   - A "Get Your Results" button is implemented, which triggers the prediction process when clicked.

3. **Prediction and Results Display**:
   - Upon clicking the predict button, a loading window appears, simulating the processing time.
   - User inputs are collected from the GUI, processed, and converted into a format suitable for prediction.
   - The model predicts the user's wage based on their inputs. The prediction is compared with the dataset to determine the percentile ranking of the predicted wage.
   - A results window is then displayed, showing the predicted wage, its percentile, and additional information like a link to a detailed wage report for the user's field, leveraging the job data loaded earlier.

4. **Interactive Elements**:
   - The application includes interactive elements like clickable links that open in the user's default web browser, allowing them to view detailed reports.
   - The application ensures a good user experience by dynamically adjusting window sizes and positions based on the user's screen dimensions.

5. **Closure and Clean-up**:
   - A close button allows users to exit the results window and the main application cleanly.

In [5]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from xgboost import XGBRegressor
import json

# Load and prepare data
data = pd.read_csv('/Users/a1234/Desktop/workspace/Employment_Analysis_and_Recommendation_System_Based_on_NLP_and_Data_Modeling/data/processed_wage_sample_data.csv')
data.ffill(inplace=True)  # Fill missing values forward

# Assuming 'Salary' is the target and the rest are features
X = data.drop('Salary', axis=1)
y = data['Salary']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)  # Fit scaler to training data

# Define the model with best parameters
best_params = {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 0.9}
model = XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(scaler.transform(X_train), y_train)  # Train the model

# Load Job Data
with open('/Users/a1234/Desktop/workspace/Employment_Analysis_and_Recommendation_System_Based_on_NLP_and_Data_Modeling/data/Cleaned_Job_Data.json', 'r') as f:
    job_data = json.load(f)

# GUI Application
class WagePredictorApp:
    def __init__(self, master):
        self.master = master
        master.title("Employment Recommendation System")

        # Labels and Entry widgets for user inputs
        labels = ['Gender', 'Length of Time on the Job (in months)', 'Age', 
                  'Years of Education Completed', 'Years of Work Experience', 
                  'Employment Category', 'Field of Employment', 'US Citizenship', 
                  'Marital Status', 'Union Membership']
        self.entries = {}
        
        for i, label in enumerate(labels):
            ttk.Label(master, text=label).grid(row=i, column=0, sticky='w', padx=5, pady=5)
            if label == 'Employment Category' or label == 'Gender' or label == 'US Citizenship' or label == 'Marital Status' or label == 'Union Membership':
                # Dropdown menu for Employment Category
                if label == 'Employment Category':
                    values = ['Clerical', 'Sales', 'Service', 'Professional', 'Manager']
                elif label == 'Gender':
                    values = ['Male', 'Female', 'Other']  # Added 'Other' option
                elif label == 'US Citizenship':
                    values = ['Yes', 'No']
                elif label == 'Marital Status':
                    values = ['Married', 'Not married']
                elif label == 'Union Membership':
                    values = ['Yes', 'No']
                self.entries[label] = ttk.Combobox(master, values=values)
            elif label == 'Field of Employment':
                values = [job['name'] for job in job_data]
                self.entries[label] = ttk.Combobox(master, values=values)
            else:
                self.entries[label] = ttk.Entry(master)
            self.entries[label].grid(row=i, column=1, padx=5, pady=5)

        # Predict button
        self.predict_button = ttk.Button(master, text="Get Your Results", command=self.predict_wage)
        self.predict_button.grid(row=len(labels), column=0, columnspan=2, pady=10)

        # Set window size
        master.update_idletasks()  # Update the window to get correct size info
        window_width = master.winfo_reqwidth() + 20  # Add some padding
        window_height = master.winfo_reqheight() + 20  # Add some padding

        # Get screen width and height
        screen_width = master.winfo_screenwidth()
        screen_height = master.winfo_screenheight()

        # Calculate window position
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2

        # Set window position and size
        master.geometry(f"{window_width}x{window_height}+{x}+{y}")

    def predict_wage(self):
        # Display loading window
        loading_window = tk.Toplevel(self.master)
        loading_window.title("Loading")

        # Get screen width and height
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()

        # Calculate window position
        window_width = 200
        window_height = 100
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2

        # Set window position
        loading_window.geometry(f"{window_width}x{window_height}+{x}+{y}")

        loading_label = ttk.Label(loading_window, text="Processing, please wait...")
        loading_label.pack(pady=20, padx=10)

        # Delay 5 seconds before running prediction and showing results
        loading_window.after(5000, lambda: self.run_prediction(loading_window))


    def run_prediction(self, loading_window):
        # Collect user input
        input_data = {
            'Sex': 0 if self.entries['Gender'].get() == 'Male' or self.entries['Gender'].get() == 'Other' else 1,  # Modified to handle 'Other'
            'Time': int(self.entries['Length of Time on the Job (in months)'].get()),
            'Age': int(self.entries['Age'].get()),
            'Edlevel': int(self.entries['Years of Education Completed'].get()),
            'Work': int(self.entries['Years of Work Experience'].get()),
            'Jobcat': self.entries['Employment Category'].get(),
            'Race': 0 if self.entries['US Citizenship'].get() == 'Yes' else 1,  # Modified to handle 'Yes'
            'Marr': 1 if self.entries['Marital Status'].get() == 'Married' else 0,
            'Union': 1 if self.entries['Union Membership'].get() == 'Yes' else 0
        }

        # Convert employment category to numerical value
        jobcat_mapping = {'Clerical': 1, 'Sales': 2, 'Service': 3, 'Professional': 4, 'Manager': 5}
        input_data['Jobcat'] = jobcat_mapping[input_data['Jobcat']]

        input_df = pd.DataFrame([input_data], columns=X_train.columns)
        input_scaled = scaler.transform(input_df)
        prediction = model.predict(input_scaled)[0]

        # Calculate the percentage of predicted wage in the dataset
        data['Predicted'] = model.predict(scaler.transform(X))  # Predict wages for all data
        percentile = (data[data['Predicted'] <= prediction].shape[0] / data.shape[0]) * 100

        # Get job data based on user's field of work
        selected_job = next(job for job in job_data if job['name'] == self.entries['Field of Employment'].get())

        # Close loading window
        loading_window.destroy()

        # Open a new window to show the result
        result_window = tk.Toplevel(self.master)
        result_window.title("Results")

        # Calculate window position
        x = (self.master.winfo_screenwidth() - 400) // 2
        y = (self.master.winfo_screenheight() - 200) // 2

        # Set window position
        result_window.geometry(f"600x210+{x}+{y}")

        result_label_1 = ttk.Label(result_window, text=f"Your Forecasted Annual Wage: $ {selected_job['Annual Wage 50']}")
        result_label_1.pack(pady=10, padx=10)
        
        result_label_2 = ttk.Label(result_window, text=f"Forecasted Wage Percentile: {percentile:.2f}%")
        result_label_2.pack(pady=10, padx=10)

        # Create a clickable link for viewing wage report
        def open_link(url):
            import webbrowser
            webbrowser.open_new(url)

        # Create labels for displaying clickable link
        result_button_3 = ttk.Button(result_window, text=f"View Wage Report for Your Field: {selected_job['url']}", cursor="hand2", command=lambda: open_link(selected_job['url']))
        result_button_3.pack(pady=10, padx=10)
        result_button_3.configure(style='Hyperlink.TButton')

        result_label_4 = ttk.Label(result_window, text=f"Your Job Code in U.S. Bureau of Labor Statistics: {selected_job['code']}")
        result_label_4.pack(pady=10, padx=10)

        # Close button
        close_button = ttk.Button(result_window, text="Close", command=self.close_window)
        close_button.pack(pady=10)

        # Print success message
        print("successful!")

    def close_window(self):
        # Close the application
        self.master.quit()

# Run the application
root = tk.Tk()
app = WagePredictorApp(root)
root.mainloop()


successful!



KeyboardInterrupt

